# Calculating a histogram

In this exercise we are going to learn how to write a computer program that estimates the probability mass function for an unknown discrete random variable.  We are then going to modify this idea so that we can also esimtate probability densities and thus work with unkown continuous random variables.  Once this is done we will look at sums of random variables, which will bring us to our first encounter with the central limit theorem.  

# Revision

In a previous exercise we introduced the law of large numbers:

$$
\lim_{n \rightarrow \infty} P\left( \left|\frac{S_n}{n} - \mathbb{E}(X) \right| < \epsilon \right) = 0 \qquad \textrm{where} \qquad S_n = X_1 + X_2 + X_3 + \dots
$$

In this expression $X_1$, $X_2$, $X_3$ and so on are all independent and identically distributed random variables.  In other words, these variables are the (random) outcomes from a set of identical experiments that have been performed.  The quantity $S_n$ is, therefore, the sum of all these random variables and in the expression on the left this is divided by the number of experiments to give us the mean.  The symbol $\mathbb{E}(X)$ is the expectation of the random variable.  For the discrete random variables we have looked at this far the expectation is given by the following summation:

$$
\mathbb{E}(X) = \sum_{i=0}^\infty x_i f_X(x_i) 
$$

In the previous exercise we showed that we could exploit the law of large numbers and estimate the expectation for a random variable by generating a large number of random variables from a distribution.  We then added these $N$ random variables together and divided by $N$ in order to get our estimate of the expectation.  The essence of this exercise is to exploit the same idea to estimate the probability mass function.  Now though instead of calculating the expectation for the random variable we calculate a set of expectation values of the form:

$$
g_X(x_i) = \mathbb{E}[\delta( X- x_i)]
$$

where the $\delta$ in the above is the following function:

$$
\delta(x) =
\begin{cases}
1 & \textrm{if} \quad x=0 \\
0 & \textrm{otherwise}
\end{cases}
$$

This expectation thus measures the fraction of times that the random variable $X$ is equal to $x_i$.  If we do this for each of the possible values that $X$ can take we get an estimate of the probability mass function.  This is the basis of calculating a histogram a technique that you have most certainly encountered before.  Can you see why?

# Some tools you will need

As always I have written you some functions that you will need in order to complete this exercise so press shift and enter on the cell below now in order to load them.

In [ ]:
import math
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rc
from IPython.display import HTML
import random
rc('animation', html='none')

class plotobj(object):
    def __init__(self,xmin,xmax):
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(xmin, xmax), ylim=(0, 1))
        
    def setup(self):
        self.line1, = self.ax.plot([],[],linestyle=':', marker='o', color='b')
        return self.line1
    
    def run(self,data):
        xdata,ydata = data
        self.ax.set_ylim(0,max(ydata))
        self.line1.set_data( xdata, ydata )
        return self.line1
    
def dynamicplot( ngen, operation, xmin, xmax, myvar ):
    myplot = plotobj(  xmin, xmax )
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=operation( ngen, myvar ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False)

# Constructing a histogram for a discrete random variable



For this first exercise we are going the experiment we are going to do is going to involve rolling two fair dice and taking the sum of the two outcomes.  As always though these dice will be virtual.  Use the cell below to write a program that returns the sum from a pair of fair dice rolls.  Call your program gendice and check it is working correctly.  If the program is working correctly the output should be a random integer, $2 \le X \le 12$.  Run it a few times to double check that this. The output should be random each time. 

We are now going to estimate the probability mass function for the random number generator you have just written by making use of some code that I have written below.  The code below will calculate the histogram you require and will give you a dynamic plot that shows how the histogram changes as more and more points are generated.  Press shift and enter on the cell below to run this calculation now. 

In [ ]:
class histoclass(object) :
    def __init__( self, xmin, xmax, nbins ):
        self.xmin=xmin
        self.xmax=xmax
        self.delr = (xmax-xmin) / float(nbins-1)
        self.xdata = []
        self.ycount = []
        for i in range(0,nbins) :
            self.xdata.append( xmin+i*self.delr )
            self.ycount.append(0)
            
    def buildHistogram( self, ngen, myvar ) :
        cnt=0
        while cnt<ngen :
            cnt += 1
            X = myvar()
            nbin = int(X - self.xmin)
            self.ycount[nbin] += 1
            tdata = []
            for dat in self.ycount :
                tdata.append( dat / cnt )
            yield self.xdata, tdata
        

myhisto = histoclass( 2., 12., 11 )
dynamicplot( 1000, myhisto.buildHistogram, myhisto.xmin, myhisto.xmax, gendice )

Now that we have code to generate a histogram for rolling two dice take some time to experiment with the above code by running it multiple times.  The outputs you get on two successive runs of the above code are never the same.  Why is this?  Write about this in your notes on this exercise.  In addition, consider the following questions and write answers to them in your report:

- In the program I draw a dotted lines between the points in my histogram because it makes the animation look more appealing.  In actuality I should not draw this line.  Can you explain why?  
- What is the most likely outcome of this experiment in which two dice are rolled?  Is your histogram always peaked at this value?  If it is not why is it not peaked at this value?
- We can determine the probability mass function for rolling two dice using the classical interpretation of probability.  Do this and draw a the resulting probability mass function in your notes.  Does the probability mass function you have drawn resemble the one that you have obtained using the program above.  If it does not how does it differ, why might we expect it to be different and what might we do to get a histogram that more closely resembles the histogram you have drawn.  

# Examining the histogram code more carefully

For this next exercise I want you to look more carefully at the code that we have just used to calculate the histogram.  As such I have copied the code above into the cell below and added comments explaining what each line in the code is doing.  

The first important thing to note here is that, in order to use the dynamic plotting tools and in order to make all the lines of code for calculating the histogram visible to you, I have had to write this program using a programming paradigm known as the object oriented paradigm ().  To be clear in the programs we have written thus far we have used python as if it is a procedural language so the building blocks of our programs have been variables that hold data and functions that change the values of these variables.  In the object oriented program below, by contrast, the building blocks are bundles of variables and functions called classes.  When we use this style of programming we have to do two things:

- We have to declare our class and explain what functions and variables are contained within it.  In the program below this is done by the set of lines that starts with class histoclass2 and that ends when the code is no longer indented.
- We have to create an instance of our class so as to set the intial values of all the variables within it.  In the program below this is done by the line myhisto = histoclass2( 2., 12., 11)

The reason I mention this is because it is important to understand in terms of understanding the flow of the code.  In essence everything in the function called \_\_init\_\_ is called first as this is done when the object is created.  The functions in buildHistogram are then run second.  Within buildHistogram the variables with names like self.ycount are variables that belong to the class.  Hence, the initial values for these variables are set in \_\_init\_\_.  The values of these variables are then used or modified in buildHistogram.  

In [ ]:
# This is the declaration of a new type of object (see above) Notice I had to 
# change the name of the class here compared to the original version
class histoclass2(object) :
    # This is the so called constructor for the object.  This function is called first.
    # There are four arguments passed to this function.  
    # self you don't need to worry about
    # xmin=lowest possible value that the random variable can take
    # xmax=largest possible value that the random variable can take
    # nbins=number of values between xmin and xmax and including xmin and xmax that the random variable can take
    def __init__( self, xmin, xmax, nbins ):
        # These two lines ensure that xmin and xmax are stored in the class so that we can 
        # access these values later in buildHistogram.  
        self.xmin=xmin
        self.xmax=xmax
        # We will see the importance of this parameter later when we learn to deal with 
        # continuous random variables.  For the time being, though, all you need to know is 
        # that xmin, xmax and nbins are set correctly this will be equal to one. 
        self.delr = (xmax-xmin) / float(nbins-1)
        # Create an empty list, which will eventually hold the set of possible values the random variable can take 
        self.xdata = []
        # Create an empty list to hold the number of instances of a particular x values
        self.ycount = []
        # This loop adds one item for each of the possible values the random variable can take to the lists 
        # created above.
        for i in range(0,nbins) :
            # Add the values that the random variable can take to the list xdata.
            self.xdata.append( xmin+i*self.delr )
            # Set the counter for this particular xvalue equal to zero.
            self.ycount.append(0)

    # This builds our histogram object.  This function is called second.
    # There are three arguments passed to this function.
    # self you don't need to worry about.
    # ngen = number of random variables to generate
    # myvar = function that generates the random variables
    def buildHistogram( self, ngen, myvar ) :
        cnt=0
        # This while condition together with the fact that we add one to cnt on each 
        # pass through the loop ensures that we only generate ngen random variables
        while cnt<ngen :
            cnt += 1
            # Generate an instance of the random variable.  For our two dice example this will be a number 2 <= X <=12
            X = myvar()
            # Notice that, although our random variable can take values 2,3,4,5,6,7,8,9.10,11,12
            # the indexes for our counter list (self.ylist) are 0,1,2,3,4,5,6,7,8,9,10
            # We can thus go from the value of the random variable to the index of self.ylist
            # by subtracting two from the random variable we generated.  This is what is done
            # on this line.
            nbin = int(X - self.xmin)
            # We now add one count to the correct counter.  To be clear for our two dice example the value of 
            # self.ycount[nbin] tells us the number of times that the random variable has
            # been equal to nbin+2 thus far
            self.ycount[nbin] += 1
            # This last part converts the counts of the number of times each number has appeared into a measure
            # of the fraction of time each number has appeared.  We do this by dividing by the total number of 
            # random variables we have generated cnt.
            tdata = []
            for dat in self.ycount : # This command is telling Python to loop over all the items in the list self.ycount
                tdata.append( dat / cnt )
            yield self.xdata, tdata
        
# Here we create an instance of the histoclass2 class.  The three numbers required to create this
# instance are:
# The minimum value that the random variable can take = 2
# The maximum value that the random variable can take = 12
# The number of integer values between these two extremes (including the two extremes) = 11 
myhisto2 = histoclass2( 2., 12., 11 )
# This plots our histogram using the dynamic plot routine that is by now hopefully familiar
# In this version the arguments are:
# The number of instances of the variable that we would like to generate = 1000
# The function that generates the data to be plotted = myhisto2.buildHistogram
# The lower bound for the xaxis = myhisto2.xmin (lowest value that the random variable can take)
# The upper bound for the yaxis = myhisto2.xmax ( highest value that the random variable can take)
# The function that generates the random variables = gendice
dynamicplot( 1000, myhisto2.buildHistogram, myhisto2.xmin, myhisto2.xmax, gendice )

To double check that we have understood everything lets work through the mathematics that the above code is performing.  For our two dice example we are essentially estimating the values of 11 expectation values using the law of large numbers:

$$
\lim_{n \rightarrow \infty} P\left( \left|\frac{S_n}{n} - \mathbb{E}(X) \right| < \epsilon \right) = 0 \qquad \textrm{where} \qquad S_n = X_1 + X_2 + X_3 + \dots
$$

In other words, we are generating 11 separate sums, $S_n$, of identically distributed random variables, $g_i(X)$, and inserting these into the expression above.  What are these random variables (or more accurately what are these functions of a random variable)?  Well they are the following:

$$
g_i(X) = \begin{cases}
1 & \textrm{if} \quad X=i \\
0 & \textrm{otherwise}
\end{cases}
$$

In other words the $i$th of our 11 random variables is a Bernoulli random variables that is one if the random variable $X$ (which we get by rolling the two dice) is equal to $i$ and zero otherwise.  Obviously, in our two dice example $2 \le i \le 12$.  Now the law of large numbers tells us that the series $\frac{g_i(X_1) + g_i(X_2) + g_i(X_3) + g_i(X_4) + \dots + g_i(X_n)}{n}$ will converge to the expectation $\mathbb{E}[g_i(X)]$ if we generate a sufficiently large number of random variables, $n$.  To be clear this is what we saw in the exercises on the law of large numbers.  Furthermore, we know that the probability mass function of a Bernoulli random variable is:

$$
f_Y(x) = P(Y=x) = \begin{cases}
1-p & \textrm{if} \quad x=0 \\
p & \textrm{if} \quad x=1 \\
0 & \textrm{otherwise}
\end{cases}
$$

and hence that the expectation of this random variable is just $p$.  In other words, the expectation (which we can estimate by exploiting the law of large numbers) is equal to the probability that our random variable, $g_i(X)$, is equal to one.  The function for $g_i$ above tells us that $g_i$ is only equal to one if the random variable $X=i$.  Consequently, the expectation for the Bernoulli random variable that we estimate using the law of large numbers is an estimate of the probability of getting $X=i$, where $X$ is our original random variable which measures the probability of getting a particular outcome when we roll two dice. 

Hopefully having read all this you now understand the logic of this program.  If so try the following exercises.

- Modify the code above so that it calculates the histogram for rolling a single fair die.  Before running the code think about what this probability mass function should look like.  In your notes write about how your estimate compares with your expectations?
- Modify the code above so that it calculates the histogram for rolling three dice.  Again think about what the probability mass function should look like before you run the code and write about how your esimtate compares with your expectations.
- Write a program to generate the outcome from a roll of $N$ dice, where $N$ is a parameter that you provide as input to the code.  Use your new random number generator and the histogram plotting tools provided to see how the histograms you obtain change as you increase the number of dice that are rolled.  In your notes on this exercise describe what you observe.

# Calculating histograms for continuous random variables

In the exercises you have just completed we computed an estimate for the probability mass function for a discrete random variable by exploiting what we have learnt about the law of large numbers.  In our two dice example where our random variable gave an outcome $2 \le X \le 12$ we saw that this was simply a matter of estimating 11 expectations $\mathbb{E}[g_i(X)]$.  Furthermore, we saw that we needed to calculate one expectation for each of the possible value that the random variable could take.  In this next exercise we would like to take this idea a step further by considering how to proceed when the random variable is continuous.  Hopefully, you can imediately see a problem here - a continuous random variable can take an infinite number of possble values.  To proceed as we did in the previous section we would thus have to calculate an infinite number of expectations.  This is something that we obviously can not do so we must proceed by a different route.  This new route works as follows.  Instead of taking the following function of our random variable $X$:

$$
g_i(X) = \begin{cases}
1 & \textrm{if} \quad X=i \\
0 & \textrm{otherwise}
\end{cases}
$$

we calculate the following integral involving our random variable, $X$:

$$
\int_{a}^b \delta(X-y) \textrm{d}y = 
\begin{cases} 
1 & \textrm{if} \quad a \le X \lt b \\
0 & \textrm{otherwise}
\end{cases}
$$

and estimate expectation values of the form:

$$
\mathbb{E}\left[ \int_{a}^b \delta(X-y) \textrm{d}y \right]
$$

by exploiting the law of large numbers.  In other words we divide the range of values the random variable can take into a finite number of segments of equal length.  We then generate multiple random variables and count the number of times the values of these random variables fall in each of these discrete ranges.  

Given the explanation above I would like you to modify the code in the following box so that it plots an estimate (obtained by generating multiple uniform random variables) of the "probability density function" for a uniform random variables that has a value between 0 and 1 (I will explain why I have put quotation marks around probability density function later).  You only need to change the two lines I have indicated to get this to work so think carefully before changing things.  Also before you run the code think about what you expect your histogram to look like so that you can compare your expectation with what you actually obtain.

In [ ]:
def mycrandom() :
    # You will need to change this line here - what random variable are we using.
    # This is currently just returning a one, which is not really very random is it?
    return 1 

class chistoclass(object) :
    def __init__( self, xmin, xmax, nbins ):
        self.xmin=xmin
        self.xmax=xmax
        # The self.delr variable will now prove critical - using the symbols in the description above this 
        # is equal to b-a.
        self.delr = (xmax-xmin) / float(nbins-1)    
        self.xdata = []
        self.ycount = []
        for i in range(0,nbins) :
            self.xdata.append( xmin+i*self.delr )
            self.ycount.append(0)
            
    def buildHistogram( self, ngen, myvar ) :
        cnt=0
        while cnt<ngen :
            cnt += 1
            X = myvar()                  
            nbin = int(X - self.xmin)     # You will need to change this line here 
            self.ycount[nbin] += 1
            tdata = []
            for dat in self.ycount :
                tdata.append( dat / cnt )
            yield self.xdata, tdata
        

mychisto = chistoclass( 0, 1, 101 )
dynamicplot( 1000, mychisto.buildHistogram, mychisto.xmin, mychisto.xmax, mycrandom )

Having understood how we write a program to esimtate the histogram for a continuous random variable lets now change the random variable we are generating to something more interesting and do a little investigation.  Do the following experiments.

- Use as your random variable $\frac{X + Y}{2}$ where $X$ and $Y$ are both uniform random variables that take values between 0 and 1.  In your notes describe the shape you expected this histogram to have before you ran the calculation and the shape that it actually has.  Was the final shape similar or different to the shape you expected?
- Use as your random variable $\frac{X + Y + Z}{3}$ where $X$, $Y$ and $Z$ are all uniform random variables that take values between 0 and 1.
- Now try $\frac{1}{N} \sum_{i=1}^N X_i$ where all the $X_i$ values are uniform random variables that take values between 0 and 1.  Try generating these histograms for a range of different $N$ values up to 200.  In your notes give a short description of how the the shape of your histogram changes as $N$ is increased.  Incidentally the shape of this curve should be reasonaly familiar - write in your notes what shape this curve has.

After you have completed these investigations watch the following video on the central limit thoerem (https://www.youtube.com/watch?v=-XJe3s_BCKw) and in your notes explain how the results you have obtained by performing experiments on calculating the histograms for discrete and continuous random variables are in agreement with the predictions of the central limit theorem.  As you write this part you will need to focus on the statement of the theorem only.  Keep it as short as you can and stay on point - there are no prizes for length.  Also note that you do not need to write anything about the estimation of error bars, which is described in the later parts of the video.  

# On estimating probability densities by calculating histograms

In the lectures and videos I have given you on this course I have stated that the probability density function, $f_X(x)$ for a continuous random variable is related to the cumulative probabiltiy distribution function, $F_X(x)$, of the variable by:

$$
f_X(x) = \frac{\textrm{d}F_X(x)}{\textrm{d}x}
$$

In the exercises above we have asserted that we can estimate the probability denisty function for a random variable by calculating a set of expectation values of the form:

$$
\mathbb{E}\left[ \int_{a}^b \delta(X-y) \textrm{d}y \right]
$$

where:

$$
\int_{a}^b \delta(X-y) \textrm{d}y = 
\begin{cases} 
1 & \textrm{if} \quad a \le X \lt b \\
0 & \textrm{otherwise}
\end{cases}
$$

We then state that the probability density function, $f_X(x)$ is equal to $\mathbb{E}\left[ \int_{a}^b \delta(X-y) \textrm{d}y \right]$ for all values of $x$ between $a$ and $b$.  Given that we do this for multiple values of $a$ and $b$ and that all these line segments are adjancent can you:

- Sketch a graph in your notes that indicates what the esimtate of the probability density function would look like if we were sampling from a normal random variable. 
- Sketch a graph in your notes showing what the integral of the function you just drew (note this integral is notionally cumulative probability distribution) would look like 

Given your diagrams answer the following questions:

- The estimate of the cumulative probability distribution that you obtain by integrating the probability density that we obtain through this method is <b>not</b> the true cumulative probability distribution function.  Justify this statement.
- Are there points on our the estimate of the cumulative probability distribution function that we obtained by integrating our histogram where the value of this approximate object is the same as the value of the true cumulative probability distribution function.  If so where are these points.  When answering this question assume that we have sufficient statistics to obtain the true distribution.
- Given these considerations if you were to repeat the exercises on the continuous random variables how would you change the methods you use to plot the sampled data to reflect the concerns that we have highlighted in this final section.  This might involve simply changing the manner in which graphs are displayed or it might involve changing the methods used to construct the histogram.